In [10]:
# ! [ -e /content ] && pip install -Uqq fastbook
# import fastbook
# fastbook.setup_book()
# #hide
# from fastai.vision.all import *
# from fastbook import *

# matplotlib.rc('image', cmap='Greys')

In [1]:
import torch
from torch import nn

In [29]:
from torchvision import datasets
from torchvision.transforms import ToTensor, ToPILImage
from torch.utils.data import DataLoader, Subset

In [3]:
datasets.MNIST

torchvision.datasets.mnist.MNIST

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cuda device


In [5]:
class NossonNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(28*28, 1024), nn.ReLU(),
            nn.Linear(1024, 512), nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.layers(x)
        return logits


In [6]:
model = NossonNet().to(device)

In [7]:
print(model)

NossonNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss()

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [10]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

In [11]:
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [12]:
bs = 64
train_dl = DataLoader(training_data, batch_size=(bs))
test_dl = DataLoader(test_data, batch_size=(bs))

In [13]:
for X, y in test_dl:
    print(f"x shape: {X.shape}")
    print(f"y shape: {y.shape} {y.dtype}")
    break

x shape: torch.Size([64, 1, 28, 28])
y shape: torch.Size([64]) torch.int64


In [14]:
training_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [44]:
sample_idx = 7
for batch, (X, y) in enumerate(train_dl):
    print(f'batch number: {batch}')
    print(f'X shape: {X.shape}')
    print(f'y shape: {y.shape}')
    print(f'sample:  {y[sample_idx]}')
    ToPILImage()(X[sample_idx]).show()
    break

batch number: 0
X shape: torch.Size([64, 1, 28, 28])
y shape: torch.Size([64])
sample:  3


In [30]:
ToPILImage

torchvision.transforms.transforms.ToPILImage

In [15]:
def train(dataloader, model, loss_fn, optimizer): # trains a single epoch
    size = len(dataloader.dataset)
    model.train() # set model to train mode - needed for accumulating batch norms averages or training augmentations
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model.forward(X)
        loss = loss_fn(pred, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [16]:
def test(dataloader, model, loss_fn): # tests a single epoch
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    model.eval() # set to eval mode - use accumulated batch norm avg instead of aggregating, no augmentation (unless applying Test Time Augmentation)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

            

In [17]:
epochs = 5


## Train

In [18]:
for t in range(epochs):
    print(f"Epoch: {t+1} \n-------------")
    train(train_dl, model, loss_fn, optimizer)
    test(test_dl, model, loss_fn)

Epoch: 1 
-------------
loss: 2.309338  [   64/60000]
loss: 0.246802  [ 6464/60000]
loss: 0.191325  [12864/60000]
loss: 0.234399  [19264/60000]
loss: 0.135810  [25664/60000]
loss: 0.347400  [32064/60000]
loss: 0.124021  [38464/60000]
loss: 0.245539  [44864/60000]
loss: 0.272951  [51264/60000]
loss: 0.180539  [57664/60000]
Test Error: 
 Accuracy: 96.0%, Avg loss: 0.128952 

Epoch: 2 
-------------
loss: 0.084376  [   64/60000]
loss: 0.089544  [ 6464/60000]
loss: 0.108369  [12864/60000]
loss: 0.100593  [19264/60000]
loss: 0.051444  [25664/60000]
loss: 0.196390  [32064/60000]
loss: 0.035104  [38464/60000]
loss: 0.113832  [44864/60000]
loss: 0.096552  [51264/60000]
loss: 0.092924  [57664/60000]
Test Error: 
 Accuracy: 97.3%, Avg loss: 0.096789 

Epoch: 3 
-------------
loss: 0.080317  [   64/60000]
loss: 0.047311  [ 6464/60000]
loss: 0.034922  [12864/60000]
loss: 0.103515  [19264/60000]
loss: 0.048520  [25664/60000]
loss: 0.075342  [32064/60000]
loss: 0.017659  [38464/60000]
loss: 0.112907

In [22]:
training_data[0][0].shape, training_data[0][0], training_data[0][1]

(torch.Size([1, 28, 28]),
 tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000

In [181]:
nn.Flatten()(training_data[0][0]).shape

torch.Size([1, 784])

### Test Overfitting:

In [20]:
train_subset = Subset(training_data, list(range(50)))
test_subset = Subset(test_data, list(range(50)))
subset_train_loader = DataLoader(train_subset, batch_size=(10))
subset_test_loader = DataLoader(test_subset, batch_size=(10))

coopppy = NossonNet().to(device)

In [90]:
opt = torch.optim.Adam(coopppy.parameters(), lr=1e-3)

In [91]:
for t in range(epochs*10):
    print(f"Epoch: {t+1} \n-------------")
    train(subset_train_loader, coopppy, loss_fn, opt)
    test(subset_test_loader, coopppy, loss_fn)

Epoch: 1 
-------------
loss: 2.313771  [   10/   50]
Test Error: 
 Accuracy: 46.0%, Avg loss: 2.062582 

Epoch: 2 
-------------
loss: 1.731281  [   10/   50]
Test Error: 
 Accuracy: 56.0%, Avg loss: 1.652768 

Epoch: 3 
-------------
loss: 1.010127  [   10/   50]
Test Error: 
 Accuracy: 70.0%, Avg loss: 1.230118 

Epoch: 4 
-------------
loss: 0.410080  [   10/   50]
Test Error: 
 Accuracy: 70.0%, Avg loss: 0.968483 

Epoch: 5 
-------------
loss: 0.131951  [   10/   50]
Test Error: 
 Accuracy: 64.0%, Avg loss: 0.885906 

Epoch: 6 
-------------
loss: 0.026182  [   10/   50]
Test Error: 
 Accuracy: 60.0%, Avg loss: 0.980376 

Epoch: 7 
-------------
loss: 0.008225  [   10/   50]
Test Error: 
 Accuracy: 62.0%, Avg loss: 1.084933 

Epoch: 8 
-------------
loss: 0.003300  [   10/   50]
Test Error: 
 Accuracy: 62.0%, Avg loss: 1.128579 

Epoch: 9 
-------------
loss: 0.001287  [   10/   50]
Test Error: 
 Accuracy: 62.0%, Avg loss: 1.145006 

Epoch: 10 
-------------
loss: 0.000633  [   1